In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pypsdm.models.input.container.grid import GridContainer


grid_path = "./simple_raw_grid"
sep = ","
grid = GridContainer.from_csv(grid_path, sep)

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'C:\\Program Files\\JetBrains\\PyCharm 2021.3.2\\jbr\\bin\\simple_raw_grid'

In [ ]:
target_grid_path = "./simple_grid"

In [ ]:
from pypsdm.plots.grid import grid_plot


grid_plot(grid)

In [ ]:
node_0 = "df97c0d1-379b-417a-a473-8e7fe37da99d"
node_1 = "6a4547a8-630b-46e4-8144-9cd649e67c07"
node_2 = "b7a5be0d-2662-41b2-99c6-3b8121a75e9e"
node_3 = "1dcddd06-f41a-405b-9686-7f7942852196"
node_4 = "e3c3c6a3-c383-4dbb-9b3f-a14125615386"

In [ ]:
from pypsdm.models.input.create.participants import create_pvs

from pypsdm.models.input.participant.pv import PhotovoltaicPowerPlants

pv_s_rated = [4, 6, 10]

data_dict = {
    "id": ["PV_NS_02", "PV_NS_03", "PV_NS_04"],
    "node": [node_2, node_3, node_4],
    "s_rated": pv_s_rated,
    "azimuth": [90, 0, -90],
    "elevation_angle": [40, 45, 30],
}

pvs = create_pvs(data_dict)

In [ ]:
from pypsdm.models.input.create.participants import create_storages


data_dict = {
    "id": ["BS_NS_02", "BS_NS_03", "BS_NS_04"],
    "node": [node_2, node_3, node_4],
    "e_storage": [1.5 * pv_power for pv_power in pv_s_rated],
}

bs = create_storages(data_dict)

In [ ]:
from pypsdm.models.input.create.participants import create_electric_vehicles
from pypsdm.models.input.participant.evs import ElectricVehicles


ev_s_rated = [10, 10, 20]
data_dict = {
    "id": ["EV_NS_02", "EV_NS_03", "EV_NS_04"],
    "node": [node_2, node_3, node_4],
    "s_rated": ev_s_rated,
    "e_storage": [80, 70, 100],
    "e_cons": [20, 16, 19],
}

evs = create_electric_vehicles(data_dict)

In [ ]:
from pipes import Template
from pypsdm.models.input.create.participants import create_ev_charging_stations
from pypsdm.models.input.participant.evcs import EvcsLocationType

data_dict = {
    "id": ["EVCS_NS_02", "EVCS_NS_03", "EVCS_NS_04"],
    "node": [node_2, node_3, node_4],
    "location_type": [
        EvcsLocationType.HOME.value,
        EvcsLocationType.HOME.value,
        EvcsLocationType.HOME.value,
    ],
    "s_rated": [10, 10, 10],
}

evcs = create_ev_charging_stations(data_dict)

In [ ]:
evs.filter_by_nodes(node_2).uuid

In [ ]:
from pypsdm.models.input.create.thermal import create_thermal_busses


data_dict = {
    "id": ["TB_NS_02", "TB_NS_03", "TB_NS_04"],
}

tbs = create_thermal_busses(data_dict)

In [ ]:
from pypsdm.models.input.create.participants import create_heat_pumps


data_dict = {
    "id": ["HP_NS_02", "HP_NS_03", "HP_NS_04"],
    "node": [node_2, node_3, node_4],
    "thermal_bus": list(tbs.uuid),
    "s_rated": [4, 3, 6],
}

hps = create_heat_pumps(data_dict)

In [ ]:
from pypsdm.models.input.create.thermal import create_thermal_houses


data_dict = {
    "thermal_bus": list(tbs.uuid),
    "id": ["TH_NS_02", "TH_NS_03", "TH_NS_04"],
    "eth_losses": [0.101, 0.121, 0.131],
    "eth_capa": [8.801, 9.121, 12.131],
}

ths = create_thermal_houses(data_dict)

In [ ]:
from pypsdm.models.input.create.participants import create_loads


load_data_dict = {
    "id": ["L_NS_02", "L_NS_03", "L_NS_04"],
    "node": [node_2, node_3, node_4],
    "s_rated": [3, 3, 3],
    "e_cons_annual": [3000, 3500, 4000],
    "load_profile": ["h0", "h0", "h0"],
}

loads = create_loads(load_data_dict)

In [ ]:
from pypsdm.models.input.container.participants import SystemParticipantsContainer
from pypsdm.models.input.participant.bm import BiomassPlants
from pypsdm.models.input.participant.em import EnergyManagementSystems
from pypsdm.models.input.participant.fixed_feed_in import FixedFeedIns
from pypsdm.models.input.participant.load import Loads
from pypsdm.models.input.participant.wec import WindEnergyConverters


participants = SystemParticipantsContainer(
    ems=EnergyManagementSystems.create_empty(),
    loads=loads,
    pvs=pvs,
    storages=bs,
    evs=evs,
    evcs=evcs,
    hps=hps,
    wecs=WindEnergyConverters.create_empty(),
    fixed_feed_ins=FixedFeedIns.create_empty(),
    biomass_plants=BiomassPlants.create_empty(),
)

In [ ]:
node_participants = participants.build_node_participants_map(grid.raw_grid.nodes)

In [ ]:
node_connected_assets = {}
for node, nodal_participants in node_participants.items():
    participant_uuids = []
    for current in nodal_participants.to_list():
        if isinstance(current, ElectricVehicles):
            continue
        participant_uuids.extend(list(current.uuid))
    node_connected_assets[node] = participant_uuids

In [ ]:
from pypsdm.models.input.create.participants import create_energy_management_systems


data_dict = {
    "id": ["EMS_NS_02", "EMS_NS_03", "EMS_NS_04"],
    "node": [node_2, node_3, node_4],
    "connected_assets": [
        node_connected_assets[node] for node in [node_2, node_3, node_4]
    ],
    "control_strategy": ["self_optimization", "self_optimization", "self_optimization"],
}

emss = create_energy_management_systems(data_dict)

In [ ]:
updated_participants = SystemParticipantsContainer(
    emss,
    participants.loads,
    participants.fixed_feed_ins,
    participants.pvs,
    participants.biomass_plants,
    participants.wecs,
    participants.storages,
    participants.evs,
    participants.evcs,
    participants.hps,
)

In [ ]:
node_participants_map = updated_participants.build_node_participants_map(
    grid.raw_grid.nodes
)
updated_grid = GridContainer(
    grid.raw_grid, updated_participants, grid.primary_data, node_participants_map
)

In [ ]:
from pypsdm.models.input.thermal.grid import ThermalGridContainer


tg = ThermalGridContainer(tbs, ths)

In [ ]:
updated_grid.to_csv(target_grid_path, include_primary_data=False)
tg.to_csv(target_grid_path)

# EV Mobility Points of Interest

In [ ]:
import pandas as pd

public_pois = pd.read_csv("./resources/public_pois.csv", index_col="uuid")

In [ ]:
from pypsdm.models.input.create.poi import create_poi_mapping


pois, poi_mappings = create_poi_mapping(updated_grid, public_pois)

poi_mappings

In [ ]:
import os


pois_path = os.path.join(target_grid_path, "pois")
if not os.path.exists(pois_path):
    os.mkdir(pois_path)
pois.to_csv(pois_path + "/poi.csv", index_label="uuid")
poi_mappings.to_csv(pois_path + "/pois_mapping.csv", index=False)